In [17]:
import shapefile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import shape
from shapely.ops import unary_union
import warnings
#warnings.filterwarnings('ignore')

In [19]:
def load_shapefile_simple(filepath):
    sf = shapefile.Reader(filepath)
    shapes = sf.shapes()
    geometries = [shape(s.__geo_interface__) for s in shapes]
    return geometries

aipolygon_geoms = load_shapefile_simple('aipolygon')
aipolygon = unary_union(aipolygon_geoms)
print(f"aipolygon: {len(aipolygon_geoms)} polygon(s) → {aipolygon.area/1e6:.2f} km²")

edensinkpolygon_geoms = load_shapefile_simple('edensinkpolygon')
edensinkpolygon = unary_union(edensinkpolygon_geoms)
print(f"edensinkpolygon: {len(edensinkpolygon_geoms)} polygon(s) → {edensinkpolygon.area/1e6:.2f} km²")

actualfloodpolygon_geoms = load_shapefile_simple('actualfloodpolygon')
actualfloodpolygon = unary_union(actualfloodpolygon_geoms)
print(f"actualfloodpolygon: {len(actualfloodpolygon_geoms)} polygon(s) → {actualfloodpolygon.area/1e6:.2f} km²")


aipolygon: 1 polygon(s) → 9.35 km²
edensinkpolygon: 1 polygon(s) → 14.45 km²
actualfloodpolygon: 106 polygon(s) → 47.52 km²


In [21]:
def calculate_metrics(poly1, poly2, name1, name2):
    """
    Calculate IoU and other metrics between two polygons
    """

    area1 = poly1.area
    area2 = poly2.area

    intersection = poly1.intersection(poly2)
    intersection_area = intersection.area
    
    union = poly1.union(poly2)
    union_area = union.area

    iou = intersection_area / union_area if union_area > 0 else 0.0
    
    # Treating poly2 as reference: TP = overlap, FP = poly1 area not in poly2, FN = poly2 area not in poly1
    TP_area = intersection_area
    FP_area = area1 - intersection_area
    FN_area = area2 - intersection_area
    
    # CSI = IoU for continuous data
    csi = TP_area / (TP_area + FP_area + FN_area) if (TP_area + FP_area + FN_area) > 0 else 0.0
    
    # POD = how much of poly2 is captured by poly1
    pod = TP_area / (TP_area + FN_area) if (TP_area + FN_area) > 0 else 0.0
    
    # FAR = how much of poly1 is outside poly2
    far = FP_area / (TP_area + FP_area) if (TP_area + FP_area) > 0 else 0.0
    
    return {
        'Name1': name1,
        'Name2': name2,
        'IoU': iou,
        'CSI': csi,
        'POD': pod,
        'FAR': far,
        'Area1_km2': area1 / 1e6,
        'Area2_km2': area2 / 1e6,
        'Intersection_km2': intersection_area / 1e6,
        'Union_km2': union_area / 1e6
    }

print("aipolygon vs edensinkpolygon")
ai_vs_eden = calculate_metrics(aipolygon, edensinkpolygon, 
                                'aipolygon', 'edensinkpolygon')

print(f"\nResults:")
print(f"  IoU:                     {ai_vs_eden['IoU']:.4f}")
print(f"  CSI:                     {ai_vs_eden['CSI']:.4f}")
print(f"  POD:                     {ai_vs_eden['POD']:.4f}")
print(f"  FAR:                     {ai_vs_eden['FAR']:.4f}")
print(f"\nArea Comparison:")
print(f"  aipolygon area:          {ai_vs_eden['Area1_km2']:.2f} km²")
print(f"  edensinkpolygon area:    {ai_vs_eden['Area2_km2']:.2f} km²")
print(f"  Overlap:                 {ai_vs_eden['Intersection_km2']:.2f} km²")

aipolygon vs edensinkpolygon

Results:
  IoU:                     0.6470
  CSI:                     0.6470
  POD:                     0.6470
  FAR:                     0.0000

Area Comparison:
  aipolygon area:          9.35 km²
  edensinkpolygon area:    14.45 km²
  Overlap:                 9.35 km²


In [23]:
# edensinkpolygon vs actualfloodpolygon
print("edensinkpolygon vs actualfloodpolygon")
eden_vs_actual = calculate_metrics(edensinkpolygon, actualfloodpolygon,
                                   'edensinkpolygon', 'actualfloodpolygon')
print(f"IoU: {eden_vs_actual['IoU']:.4f}")
print(f"CSI: {eden_vs_actual['CSI']:.4f}")
print(f"POD: {eden_vs_actual['POD']:.4f}")
print(f"FAR: {eden_vs_actual['FAR']:.4f}")

# aipolygon vs actualfloodpolygon
print("\n aipolygon vs actualfloodpolygon")
ai_vs_actual = calculate_metrics(aipolygon, actualfloodpolygon,
                                 'aipolygon', 'actualfloodpolygon')
print(f"IoU: {ai_vs_actual['IoU']:.4f}")
print(f"CSI: {ai_vs_actual['CSI']:.4f}")
print(f"POD: {ai_vs_actual['POD']:.4f}")
print(f"FAR: {ai_vs_actual['FAR']:.4f}")

edensinkpolygon vs actualfloodpolygon
IoU: 0.2458
CSI: 0.2458
POD: 0.2573
FAR: 0.1540

 aipolygon vs actualfloodpolygon
IoU: 0.1746
CSI: 0.1746
POD: 0.1779
FAR: 0.0958


In [25]:
metrics_table = pd.DataFrame({
    'Comparison': [
        'aipolygon vs edensinkpolygon',
        'edensinkpolygon vs actualfloodpolygon',
        'aipolygon vs actualfloodpolygon'
    ],
    'IoU': [
        ai_vs_eden['IoU'],
        eden_vs_actual['IoU'],
        ai_vs_actual['IoU']
    ],
    'CSI': [
        ai_vs_eden['CSI'],
        eden_vs_actual['CSI'],
        ai_vs_actual['CSI']
    ],
    'POD': [
        ai_vs_eden['POD'],
        eden_vs_actual['POD'],
        ai_vs_actual['POD']
    ],
    'FAR': [
        ai_vs_eden['FAR'],
        eden_vs_actual['FAR'],
        ai_vs_actual['FAR']
    ]
})

metrics_table = pd.DataFrame({
    'Comparison': [
        'aipolygon vs edensinkpolygon',
        'edensinkpolygon vs actualfloodpolygon',
        'aipolygon vs actualfloodpolygon'
    ],
    'IoU': [
        ai_vs_eden['IoU'],
        eden_vs_actual['IoU'],
        ai_vs_actual['IoU']
    ],
    'CSI': [
        ai_vs_eden['CSI'],
        eden_vs_actual['CSI'],
        ai_vs_actual['CSI']
    ],
    'POD': [
        ai_vs_eden['POD'],
        eden_vs_actual['POD'],
        ai_vs_actual['POD']
    ],
    'FAR': [
        ai_vs_eden['FAR'],
        eden_vs_actual['FAR'],
        ai_vs_actual['FAR']
    ]
})

print("\n" + metrics_table.to_string(index=False, float_format=lambda x: f'{x:.4f}'))




                           Comparison    IoU    CSI    POD    FAR
         aipolygon vs edensinkpolygon 0.6470 0.6470 0.6470 0.0000
edensinkpolygon vs actualfloodpolygon 0.2458 0.2458 0.2573 0.1540
      aipolygon vs actualfloodpolygon 0.1746 0.1746 0.1779 0.0958
